<a href="https://colab.research.google.com/github/Oks11111/TP_entrega_vpc2/blob/main/Clase_1_Tarea.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

>> Con el código utilizado en clase, implementar una clase de Python similar a esta. Se le deben pasar como argumentos:
* Cantidad de filtros +
* Tamaño de los filtros +
* Stride +
* Tipo de padding (“valid” o “same”) +
* Debe tener un método “forward”, al cual se le pasa la matriz de entrada y devuelve la salida. +
* Y un método “backward”, al cual se le pasa una matriz con ∂L/∂O y devuelve dos matrices, una con ∂L/∂F y otra con ∂L/∂X. +



*
filters = 8
filter_size = 3
stride = 2
pad = 1

In [ ]:
import numpy as np
import torch

In [ ]:
# Función para padding con ceros

def zero_padding(X, pad):
  """
  Argumentos:
    X: Array numpy de entrada con dimensiones (batch_size, n_C, n_H, n_W)
    pad: Entero representando la cantidad de filas y columnas que se deben agregar con ceros

  Retorna:
    X_padded: Array numpy con dimensiones (batch_size, n_C, n_H + 2*pad, n_W + 2*pad)
  """

  X_padded = np.pad(X, ((0,0), (0,0), (pad, pad), (pad, pad)), mode='constant', constant_values = (0,0))

  return X_padded

In [ ]:
# Función para realizar la operación de convolución

def convolve(X, W, b):
  """
  Argumentos:
    X: Array numpy de entrada con dimensiones (filter_size, filter_size, n_C_prev)
    W: Array numpy con los pesos de un filtro con dimensiones (filter_size, filter_size, n_C_prev)
    b: Entero con el valor de bias de la capa actual

  Retorna:
    Z: Entero con el valor del resultado
  """

  # Multiplico elemento a elemento el valor de entrada con los pesos del filtro
  aux = X * W
  # Realizo la suma de todos los elementos
  aux = np.sum(aux)
  # Le sumo el valor del bias para obtener Z
  Z = aux + float(b)

  return Z

In [ ]:
# Función para realizar el forward pass de una capa convolucional

def conv_forward(layer_input, W, b, stride, padding,pad):
  """
  Argumentos:
    layer_input: Array numpy con los valores de entrada a la capa convolucional (batch_size, n_C_prev, n_H_prev, n_W_prev)
    W: Array numpy con los pesos de los filtros utilizados en la capa actual (n_C, n_C_prev, filter_size, filter_size)
    b: Array numpy con los valores de bias utilizados en la capa actual (1, 1, 1, n_C)
    stride: Entero con el valor de stride utilizado en la capa actual.
    padding: 

  Retorna:
    Z: Array numpy con los valores de salida de la capa convolucional (batch_size, n_H, n_W, n_C)
  """

  # Obtengo las dimensiones de la entrada
  (batch_size, n_C_prev, n_H_prev, n_W_prev) = layer_input.shape

  # Obtengo las dimensiones de los filtros
  (n_C, n_C_prev, filter_size, filter_size) = W.shape

  # Calculo las dimensiones del volumen de salida de la capa actual

  if pad == 'VALID':

      n_H = int((n_H_prev + 2*padding - filter_size)/stride + 1)
      n_W = int((n_W_prev + 2*padding - filter_size)/stride + 1)

      # Inicializo el volumen de salida con ceros
      Z = np.zeros([batch_size, n_C, n_H, n_W])

      # Agrego padding con ceros al volumen de entrada
      layer_input_padded = zero_padding(layer_input, padding)
        
        
  if pad == 'SAME':
      n_H = int(n_H_prev / stride)
      n_W = int(n_W_prev / stride)
      # Inicializo el volumen de salida con ceros
      Z = np.zeros([batch_size, n_C, n_H, n_W])
      layer_input_padded = layer_input
  

  # Comienzo iterando sobre cada ejemplo del batch
  for i in range(batch_size):

    # Itero sobre el eje vertical del volumen de salida
    for h in range(n_H):
      # Calculo las coordenadas verticales de inicio y fin de la ventana sobre la que aplicaremos el filtro
      y_start = stride * h
      y_end = y_start + filter_size

      # Itero sobre el eje horizontal del volumen de salida
      for w in range(n_W):
        # Calculo las coordenadas horizontales de inicio y fin de la ventana sobre la que aplicaremos el filtro
        x_start = stride * w
        x_end = x_start + filter_size

        # Extraigo la ventana para calcular la convolucion, del volumen de entrada con padding
        slice_from_input_padded = layer_input_padded[i, :, y_start:y_end, x_start:x_end]
        
        # Itero sobre la cantidad de canales del volumen de salida
        for c in range(n_C):

          # Obtengo el valor del filtro y bias del canal correspondiente
          filter = W[c, :, :, :]
          bias = b[c]

          # Computo la operación de convolución para esta ventana
          Z[i, c, h, w] = convolve(slice_from_input_padded, filter, bias)
          
  cache = (layer_input, W, b, stride, padding)
          
  
  return Z, cache

In [ ]:
# Dimensiones de la entrada
batch_size = 10
input_height, input_width = (7, 7)
input_channels = 4

# Dimensiones de la convolucional
filters = 8
filter_size = 3
stride = 2
pad = 1
padd="VALID"

# Variables de prueba
test_array = np.random.randn(batch_size, input_channels, input_height, input_width)
W = np.random.randn(filters, input_channels, filter_size, filter_size)
b = np.random.randn(filters)
print("W.shape",W.shape)
print("b.shape",b.shape)
conv_result, cache = conv_forward(test_array, W, b, stride, pad,padd)
conv_result_pyt = torch.nn.functional.conv2d(torch.tensor(test_array), torch.tensor(W), torch.tensor(b), stride, pad)

assert(conv_result.shape == conv_result_pyt.shape)
print("Convolución: Result shape: {}".format(conv_result.shape))
print("Convolución: Result value: {}".format(conv_result[1, 1, 1, 1]))
print("Convolución: Result shape: {}".format(conv_result_pyt.shape))
print("Convolución: Result value: {}".format(conv_result_pyt[1, 1, 1, 1]))


W.shape (8, 4, 3, 3)
b.shape (8,)
Convolución: Result shape: (10, 8, 4, 4)
Convolución: Result value: -5.76483299856314
Convolución: Result shape: torch.Size([10, 8, 4, 4])
Convolución: Result value: -5.764832998563142


In [ ]:
def conv_backward(dZ, cache):   #cahe es mi Z de forward
    """
    Se implementa backward propagation para función de convolución
    
    Argumentos:
    dZ -- es el Z de salida de convolución mismo tamaño (m, n_H, n_W, n_C), m=batch_size
    cache -- Los demas valores necesarios para conv_backward(), salida de conv_forward()
    
    Retorna:

    dA_prev -- gradiente de costos al respecto de entrada a capa convolucional (layer_input), 
         numpy array of shape (m, n_H_prev, n_W_prev, n_C_prev)   

    dW -- gradiente costos al respecto de pesos de capa convolucional (W)
         numpy array of shape (f, f, n_C_prev, n_C)  

    db -- gradient of the cost with respect to the biases of the conv layer (b)
          numpy array of shape (1, 1, 1, n_C)
    """
    



    # Buscamos los parametros 
    (layer_input, W, b, stride, pad) = cache
    
    # Dimenciones de layer_input
    (m, n_C_prev, n_H_prev, n_W_prev) = layer_input.shape
    
    # Dimenciones de W
    (n_C, n_C_prev, f, f) = W.shape
  
    # Dimenciones de dZ (Z, salida de convol)
    
    (m, n_C, n_H, n_W) = dZ.shape
    
    # Inicializamos dA, dW, db con las dimenciones extraidas
    dA_prev = np.zeros((m, n_C_prev, n_H_prev, n_W_prev))                           
    dW = np.zeros((n_C, n_C_prev, f, f))
    db = np.zeros((1, 1, 1, n_C))

    # Pad layer_input y dA_prev
    A_prev_pad = zero_padding(layer_input, pad)
    dA_prev_pad = zero_padding(dA_prev, pad)
    
    for i in range(m):                       # loop a travez de batch
        
        # seleccionamos i-essimo batch
        a_prev_pad = A_prev_pad[i]
        da_prev_pad = dA_prev_pad[i]
        
        for h in range(n_H):                   # loop eje vertical
            for w in range(n_W):               # loop eje horizontal
                for c in range(n_C):           # loop a travez de canales
                    
                    # limites de "slice"
                    vert_start  = h*stride
                    vert_end    = vert_start + f
                    horiz_start = w*stride
                    horiz_end   = horiz_start + f
                    
                    # slice de a_prev_pad
                    a_slice = a_prev_pad[:,vert_start:vert_end, horiz_start:horiz_end]
                   
                    # Actualizamos los gradientes para la ventana para los filtros dados
                   
                    da_prev_pad[:,vert_start:vert_end, horiz_start:horiz_end] += W[c,:,:,:] * dZ[i, c, h, w]
                    dW[c,:,:,:] += a_slice * dZ[i, c, h, w]
                    db[:,:,:,c] += dZ[i, c, h, w]
                    
        # guardamos i-essimo batch de dA_prev, recortamos pad
        dA_prev[i, :, :, :] = da_prev_pad[:,pad:-pad, pad:-pad]
        
    assert(dA_prev.shape == (m, n_C_prev, n_H_prev, n_W_prev))
    print("dA shape=",dA_prev.shape, "db shape=",db.shape, "dW shape=",dW.shape)
    return dA_prev, dW, db

In [ ]:

np.random.seed(1)
dA, dW, db = conv_backward(conv_result, cache)
print("dA_mean =", np.mean(dA))
print("dW_mean =", np.mean(dW))
print("db_mean =", np.mean(db))
print("dA =", dA)
print("dW =", dW)
print("db =", db)

dA shape= (10, 4, 7, 7) db shape= (1, 1, 1, 8) dW shape= (8, 4, 3, 3)
dA_mean = -1.2800392027496326
dW_mean = -12.118332182254083
db_mean = 100.52056978130281
dA = [[[[-1.77150746e+01  7.58242254e+00  5.33431709e+00 ... -4.96868696e+00
    -2.25059893e+01 -3.14489530e+00]
   [-7.91106493e+00 -2.20958564e+01  1.30847485e+00 ...  1.35726232e+01
    -7.41993789e+01 -3.81062050e+01]
   [-8.79998550e+00  1.60105700e+01  1.22325441e+01 ... -2.51095930e+01
     2.84674555e+01 -7.03269091e+00]
   ...
   [-6.81818791e+00  2.04089656e+01  4.08919724e+00 ... -1.93397786e+01
     2.97245361e+01  8.36053793e+00]
   [ 3.43511178e+01 -3.08166498e+01  5.02047430e+01 ...  3.47055526e+00
    -6.12877264e+01 -5.82445265e+01]
   [ 3.90656127e+00  4.41801932e+01  2.15632564e+00 ...  2.17625294e+00
     1.56417850e+01 -1.40254420e+00]]

  [[-1.17096178e+01 -2.76736056e+00  1.80784861e+01 ... -1.30880811e+01
     3.22900428e+00 -1.01024295e+01]
   [ 9.84590010e+00  3.95783060e+01  3.84290727e+00 ... -4.31917